# Data Pipeline: Load and Clean NYC 311 and Liquor License Data

This notebook loads NYC 311 noise complaints and liquor license data, filters for Brooklyn and Staten Island (2024), cleans ZIP codes, and saves cleaned datasets.

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 2. Load Data from NYC Open Data URLs

In [ ]:
# NYC 311 Service Requests - Noise complaints
# Filter for noise complaints in 2024 for Brooklyn and Staten Island
print("Loading 311 noise complaint data from NYC Open Data...")
print("This may take a few minutes...\n")

# SOCRATA API endpoint for 311 data with filters
# Filter: Complaint Type contains 'Noise', Created Date in 2024, Borough in Brooklyn or Staten Island
url_311 = (
    "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?"
    "$where=complaint_type like '%25Noise%25' AND "
    "created_date >= '2024-01-01T00:00:00' AND "
    "created_date < '2025-01-01T00:00:00' AND "
    "(borough = 'BROOKLYN' OR borough = 'STATEN ISLAND')"
    "&$limit=50000"
)

try:
    df_311 = pd.read_csv(url_311)
    print(f"Loaded {len(df_311)} noise complaint records from 2024")
except Exception as e:
    print(f"Error loading 311 data: {e}")
    print("Attempting to use alternative method...")
    # Fallback: load all data and filter locally
    df_311 = pd.read_csv(
        "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$limit=50000&$where=complaint_type like '%25Noise%25'"
    )
    df_311['created_date'] = pd.to_datetime(df_311['created_date'])
    df_311 = df_311[
        (df_311['created_date'].dt.year == 2024) &
        (df_311['borough'].isin(['BROOKLYN', 'STATEN ISLAND']))
    ]
    print(f"Loaded {len(df_311)} noise complaint records from 2024")

In [ ]:
# NYC Liquor License Data
# Filter for active licenses in Brooklyn and Staten Island
print("\nLoading liquor license data from NYC Open Data...")
print("This may take a few minutes...\n")

# SOCRATA API endpoint for liquor licenses
# Filter for active licenses in Kings (Brooklyn) and Richmond (Staten Island) counties
url_liquor = (
    "https://data.cityofnewyork.us/resource/tjby-92xf.csv?"
    "$where=license_status = 'Active' AND "
    "(county = 'Kings' OR county = 'Richmond')"
    "&$limit=50000"
)

try:
    df_liquor = pd.read_csv(url_liquor)
    print(f"Loaded {len(df_liquor)} active liquor license records")
except Exception as e:
    print(f"Error loading liquor data: {e}")
    print("Attempting to use alternative method...")
    # Fallback: load all data and filter locally
    df_liquor = pd.read_csv(
        "https://data.cityofnewyork.us/resource/tjby-92xf.csv?$limit=50000"
    )
    df_liquor = df_liquor[
        (df_liquor['license_status'] == 'Active') &
        (df_liquor['county'].isin(['Kings', 'Richmond']))
    ]
    print(f"Loaded {len(df_liquor)} active liquor license records")

## 3. Clean ZIP Codes to Standard 5-Digit Format

In [ ]:
def clean_zip_code(zip_code):
    """
    Clean ZIP code to standard 5-digit format.
    Handles various formats like '11201', '11201-1234', etc.
    """
    if pd.isna(zip_code):
        return None
    
    # Convert to string and remove any whitespace
    zip_str = str(zip_code).strip()
    
    # Remove anything after a dash (ZIP+4 format)
    if '-' in zip_str:
        zip_str = zip_str.split('-')[0]
    
    # Remove any non-numeric characters
    zip_str = ''.join(c for c in zip_str if c.isdigit())
    
    # Pad with leading zeros if necessary
    if len(zip_str) > 0:
        zip_str = zip_str.zfill(5)
        
        # Only return if it's a valid 5-digit ZIP
        if len(zip_str) == 5:
            return zip_str
    
    return None

# Clean 311 data ZIP codes
print("Cleaning ZIP codes in 311 data...")
if 'incident_zip' in df_311.columns:
    df_311['zip_code'] = df_311['incident_zip'].apply(clean_zip_code)
    valid_311 = df_311['zip_code'].notna().sum()
    print(f"311 data: {valid_311} records with valid ZIP codes out of {len(df_311)} total")
elif 'zip_code' in df_311.columns:
    df_311['zip_code'] = df_311['zip_code'].apply(clean_zip_code)
    valid_311 = df_311['zip_code'].notna().sum()
    print(f"311 data: {valid_311} records with valid ZIP codes out of {len(df_311)} total")
else:
    print("Warning: No ZIP code column found in 311 data")

# Clean liquor license ZIP codes
print("\nCleaning ZIP codes in liquor license data...")
if 'premises_zip_code' in df_liquor.columns:
    df_liquor['zip_code'] = df_liquor['premises_zip_code'].apply(clean_zip_code)
    valid_liquor = df_liquor['zip_code'].notna().sum()
    print(f"Liquor data: {valid_liquor} records with valid ZIP codes out of {len(df_liquor)} total")
elif 'zip_code' in df_liquor.columns:
    df_liquor['zip_code'] = df_liquor['zip_code'].apply(clean_zip_code)
    valid_liquor = df_liquor['zip_code'].notna().sum()
    print(f"Liquor data: {valid_liquor} records with valid ZIP codes out of {len(df_liquor)} total")
else:
    print("Warning: No ZIP code column found in liquor data")

## 4. Filter Out Records Without Valid ZIP Codes

In [ ]:
# Keep only records with valid ZIP codes for analysis
df_311_clean = df_311[df_311['zip_code'].notna()].copy()
df_liquor_clean = df_liquor[df_liquor['zip_code'].notna()].copy()

print(f"After filtering for valid ZIP codes:")
print(f"311 noise complaints: {len(df_311_clean)} records")
print(f"Liquor licenses: {len(df_liquor_clean)} records")

## 5. Save Cleaned Data

In [ ]:
# Create output directory if it doesn't exist
output_dir = Path('../data/clean')
output_dir.mkdir(parents=True, exist_ok=True)

# Save cleaned datasets
print("Saving cleaned datasets...\n")

noise_output = output_dir / '311_noise_cleaned.csv'
liquor_output = output_dir / 'liquor_cleaned.csv'

df_311_clean.to_csv(noise_output, index=False)
print(f"Saved: {noise_output}")
print(f"  - {len(df_311_clean)} records")

df_liquor_clean.to_csv(liquor_output, index=False)
print(f"\nSaved: {liquor_output}")
print(f"  - {len(df_liquor_clean)} records")

## 6. Display First Few Rows as Confirmation

In [ ]:
print("\n" + "="*80)
print("311 NOISE COMPLAINTS - FIRST 5 ROWS")
print("="*80)
print(df_311_clean.head())

print("\n" + "="*80)
print("LIQUOR LICENSES - FIRST 5 ROWS")
print("="*80)
print(df_liquor_clean.head())

In [ ]:
# Summary statistics
print("\n" + "="*80)
print("DATA SUMMARY")
print("="*80)
print(f"\n311 Noise Complaints (2024, Brooklyn & Staten Island):")
print(f"  Total records: {len(df_311_clean)}")
if 'borough' in df_311_clean.columns:
    print(f"  By borough:")
    print(df_311_clean['borough'].value_counts().to_string())
print(f"  Unique ZIP codes: {df_311_clean['zip_code'].nunique()}")

print(f"\nLiquor Licenses (Active, Brooklyn & Staten Island):")
print(f"  Total records: {len(df_liquor_clean)}")
if 'county' in df_liquor_clean.columns:
    print(f"  By county:")
    print(df_liquor_clean['county'].value_counts().to_string())
print(f"  Unique ZIP codes: {df_liquor_clean['zip_code'].nunique()}")